In [2]:
# IMPORTANTE:
# não se esqueça de atualizar as datas [filtros e excel]
# sempre a ultima versão do banco analisada

In [3]:
ultima_analise = '2022-03-03'

In [4]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")
# puxar arquivo do banco
df = pd.read_csv('eapv_15032022_09h32.csv', sep=';', encoding='utf-8')
print(f'Banco importado: {df.shape}')
banco_completo = df.shape[0]

Banco importado: (17980, 189)


# Excluir as fichas canceladas

In [5]:
# Transformando ['Número da Notificação'] para string
df['Número da Notificação'] = df['Número da Notificação'].astype(str)
# puxar arquivo canceladas
df_canceladas = pd.read_csv('eapv_canceladas_15032022.csv', sep=';', encoding='utf-8')
# df_canceladas passa a ser apenas a coluna 'Número da Notificação' e em string
df_canceladas = df_canceladas[['Número da Notificação']].astype(str)
print(f'Banco canceladas: {df_canceladas.shape}')
banco_canceladas = df_canceladas.shape[0]
# Transformando o banco df_canceladas para uma lista notif_canceladas
notif_canceladas = df_canceladas['Número da Notificação'].to_list()

Banco canceladas: (948, 1)


In [6]:
# TIRANDO AS EXCLUÍDAS DO BANCO DE DADOS 
df = df[~df['Número da Notificação'].str.contains('(?i)' + '|'.join(notif_canceladas), na=False)]

# conferir
print(f'Banco sem as canceladas: {df.shape[0]}')
print(f'Banco completo: {banco_completo} - canceladas: {banco_canceladas} = {banco_completo - banco_canceladas}')

# Verificar se ficou alguma cancelada e excluir do banco
df[df['Situação da Notificação'] == 'Cancelado']
df = df[~df['Situação da Notificação'].str.contains('Cancelado', na=False)]

Banco sem as canceladas: 17032
Banco completo: 17980 - canceladas: 948 = 17032


# Tratar vacinas

In [7]:
# Criar df_dubias apenas com as dubias
df_dubias = df.loc[~(df['Imunobiológico (vacina)'].str.contains('(?i)Butantan') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Sinovac') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Oxford') |
             df['Imunobiológico (vacina)'].str.contains('(?i)AstraZeneca') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Covishield') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Pfizer') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Janssen'))]

# Manter no df apenas vacinas corretas 
df = df.loc[(df['Imunobiológico (vacina)'].str.contains('(?i)Butantan') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Sinovac') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Oxford') |
             df['Imunobiológico (vacina)'].str.contains('(?i)AstraZeneca') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Covishield') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Pfizer') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Janssen'))]

In [8]:
bd_vacinas_covid = df.shape[0]

print(f'Foram excluídas do banco {banco_completo - bd_vacinas_covid} fichas que contém outras vacinas que não as COVID-19')
print(f'Banco somente com vacinas COVID-19: {df.shape}')

Foram excluídas do banco 994 fichas que contém outras vacinas que não as COVID-19
Banco somente com vacinas COVID-19: (16986, 189)


.

.

A PARTIR DAQUI, É POSSÍVEL RODAR SOMENTE AS CÉLULAS DE INTERESSE

# Abaixo de 12 anos

In [30]:
# Encontrar novos casos e enviar para [helena-ramos@saude.rs.gov.br]

In [40]:
df_12_anos = df.copy()
df_12_anos = df_12_anos[df_12_anos['Idade Evento'] < 12]
df_12_anos = df_12_anos.sort_values('Idade Evento').reset_index(drop=True)
df_12_anos['Data da Notificação'] =  pd.to_datetime(df_12_anos['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_12_anos = df_12_anos[df_12_anos['Data da Notificação']>= ultima_analise]
df_12_anos = df_12_anos.loc[df_12_anos['Classificação de gravidade'].str.contains("Grave", na=False)]
print(f'Há {df_12_anos.shape[0]} casos')

In [41]:
# Gerar excel
#df_12_anos = df_12_anos[['Número da Notificação', 'Nome Completo', 'Idade Evento']]
#df_12_anos.to_excel('eapv_abaixo_12_anos_03032022.xlsx')

Há 0 casos


# Guillain Barre

In [14]:
# Encontrar novos casos e enviar para [rosemeri-cohen@saude.rs.gov.br]

In [42]:
df_guillain = df.copy()
df_guillain = df_guillain.loc[df_guillain.apply(lambda row: row.astype(str).str.contains('(?i)guill').any(), axis=1)]
df_guillain['Data da Notificação'] = pd.to_datetime(df_guillain['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_guillain = df_guillain[df_guillain['Data da Notificação']>= ultima_analise]
print(f'Há {df_guillain.shape[0]} casos')

In [43]:
# Gerar excel
df_guillain = df_guillain[['Número da Notificação', 'Nome Completo']]
df_guillain.to_excel('eapv_guillain_barre_15032022.xlsx')

Há 1 casos


# Tromboses

In [56]:
# Encontrar novos casos e enviar para [rosemeri-cohen@saude.rs.gov.br]

In [9]:
df_tromboses = df.copy()
tromboses = ['trombo', 'embolia']
df_tromboses = df_tromboses.loc[df_tromboses.apply(lambda row: row.astype(str).str.contains('(?i)' + '|'.join(tromboses)).any(), axis=1)]
df_tromboses['Data da Notificação'] = pd.to_datetime(df_tromboses['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_tromboses = df_tromboses[df_tromboses['Data da Notificação']>= ultima_analise]
print(f'Há {df_tromboses.shape[0]} casos')

Há 4 casos


In [10]:
# Gerar excel
df_tromboses = df_tromboses[['Número da Notificação', 'Nome Completo']]
df_tromboses.to_excel('eapv_tromboses_15032022.xlsx')

# Gestantes

In [75]:
# Encontrar novos casos e enviar para [helena-ramos@saude.rs.gov.br]

In [46]:
df_gestantes = df.copy()
df_gestantes = df_gestantes.loc[df_gestantes['Gestante no momento da vacinação?'].str.contains("(?i)Sim", na=False)]
df_gestantes = df_gestantes.loc[df_gestantes['Classificação de gravidade'].str.contains("Grave", na=False)]
df_gestantes['Data da Notificação'] = pd.to_datetime(df_gestantes['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_gestantes = df_gestantes[df_gestantes['Data da Notificação']>= ultima_analise]
print(f'Há {df_gestantes.shape[0]} casos')

Há 0 casos


In [11]:
# Gerar excel
#df_gestantes = df_gestantes[['Número da Notificação', 'Nome Completo']]
#df_gestantes.to_excel('eapv_gestantes_graves_03032022.xlsx')

.

.

.

## TRATAMENTO DO BANCO

Encerrados graves por municípios

In [ ]:
# bug no e-sus, mostra encerrados pelo estado na coluna dos municípios... verificar manualmente
# se encontrar algum caso, enviar email aos notificadores

In [1]:
data = '2022-02-08'

In [12]:
df_municipio = df.copy()
df_municipio = df_municipio.loc[df_municipio['Classificação de gravidade'].str.contains("Grave", na=False)]
df_municipio.dropna(subset=['Encerramento Municipal - Evento adverso'], inplace=True)
df_municipio['Data da Notificação'] = pd.to_datetime(df_municipio['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_municipio = df_municipio[df_municipio['Data da Notificação']>= data]
print(f'Há {df_municipio.shape[0]} casos')

Há 0 casos


Vacina Dúbias

In [13]:
# usar df_dubias para pesquisar vacinas dubias
# fazer isso manualemte, por enquanto
# caso encontre, enviar e-mail aos notificadores

In [83]:
# Gerar excel
df_municipio.to_excel('encerrados_por_municipio.xlsx')

Há 0 casos


In [ ]:
# gerar excel
#df_dubias.to_excel('vacinas_dubias.xlsx')

Idade inconscistente

In [14]:
# fazer idade inconsc aqui

In [ ]:
# PARA FAZER
# Filtrar apenas colunas de interesse. Identificar quais...
# Verificar casos de SARC em Imunobiológico(vacina) e gerar xlsx